In [1]:
import torch
from vit_pytorch.efficient import ViT
from performer_pytorch import Performer
from __future__ import print_function
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
device = torch.device("cuda")

In [3]:
performer = Performer(
    dim = 512,
    depth = 1,
    heads = 8,
    causal = False,
    dim_head = 64
)

model = ViT(
    dim = 512,
    image_size = 28,
    patch_size = 7,
    num_classes = 10,
    transformer = performer
)

model = model.to(device)
img = torch.randn(1, 3, 28, 28).to(device) # your high resolution picture
with torch.no_grad():
    output = model(img) # (1, 1000)

In [4]:
performer

Performer(
  (net): SequentialSequence(
    (layers): ModuleList(
      (0): ModuleList(
        (0): PreLayerNorm(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fn): SelfAttention(
            (fast_attention): FastAttention(
              (kernel_fn): ReLU()
            )
            (to_q): Linear(in_features=512, out_features=512, bias=True)
            (to_k): Linear(in_features=512, out_features=512, bias=True)
            (to_v): Linear(in_features=512, out_features=512, bias=True)
            (to_out): Linear(in_features=512, out_features=512, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (1): PreLayerNorm(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fn): Chunk(
            (fn): FeedForward(
              (w1): Linear(in_features=512, out_features=2048, bias=True)
              (act): GELU(approximate='none')
              (dropout): Dropout(p=0.0, inplac

In [44]:
lr = 1e-2
gamma = 0.7
optimizer = optim.AdamW(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.expand(-1, 3, -1, -1)
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.expand(-1, 3, -1, -1)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [46]:
from torchvision import datasets, transforms
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('./data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('./data', train=False, download=True,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=16)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=16)